# S&P500 trend follwing on mt5 

idea for this research is to see what the best way would be to deploy algo that would time the market on us500.

1. Using bollinger bands 
2. Using emas 
3. Using engulfing candles or pinbar candles


### Result



In [6]:
'''
Start                     2024-01-02 05:45:00
End                       2024-03-28 23:45:00
Duration                     86 days 18:00:00
Exposure Time [%]                   73.943294
Equity Final [$]                      14888.2
Equity Peak [$]                       14963.6
Return [%]                             48.882
Buy & Hold Return [%]                9.992667
Return (Ann.) [%]                  390.245473
Volatility (Ann.) [%]               250.15073
Sharpe Ratio                         1.560041
Sortino Ratio                       21.004768
Calmar Ratio                        30.294429
Max. Drawdown [%]                  -12.881757
Avg. Drawdown [%]                   -0.690614
Max. Drawdown Duration       13 days 22:00:00
Avg. Drawdown Duration        0 days 14:26:00
# Trades                                   90
Win Rate [%]                        33.333333
Best Trade [%]                       1.172458
Worst Trade [%]                     -0.523848
Avg. Trade [%]                       0.087714
Max. Trade Duration           4 days 04:45:00
Avg. Trade Duration           0 days 17:03:00
Profit Factor                        1.607989
'''

'\nStart                     2024-01-02 05:45:00\nEnd                       2024-03-28 23:45:00\nDuration                     86 days 18:00:00\nExposure Time [%]                   73.943294\nEquity Final [$]                      14888.2\nEquity Peak [$]                       14963.6\nReturn [%]                             48.882\nBuy & Hold Return [%]                9.992667\nReturn (Ann.) [%]                  390.245473\nVolatility (Ann.) [%]               250.15073\nSharpe Ratio                         1.560041\nSortino Ratio                       21.004768\nCalmar Ratio                        30.294429\nMax. Drawdown [%]                  -12.881757\nAvg. Drawdown [%]                   -0.690614\nMax. Drawdown Duration       13 days 22:00:00\nAvg. Drawdown Duration        0 days 14:26:00\n# Trades                                   90\nWin Rate [%]                        33.333333\nBest Trade [%]                       1.172458\nWorst Trade [%]                     -0.523848\nAvg. Trade

In [7]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
mt5.initialize()

True

In [8]:
symbol = 'SP500USD'
timeframe = mt5.TIMEFRAME_M15
start_date = datetime(2023,1,1)
end_time = datetime.now()

def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.columns = df.columns.str.capitalize()
    return df

us500 = get_data(symbol, timeframe, start_date, end_time)
us500.dropna(inplace=True)


### Main logic 

This logic is based on the lower bb and rsi. the trades are taken when the price closes above the lower Bb only when the RSI s below a certain value, which in this case 40 is used.

In [9]:
def generate_signal(data):
    data['Lower_BB'] = ta.volatility.bollinger_lband(data['Close'], window=20)
    data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
    data['Signals'] = 0
    
    for i in range(1, len(data)):
        
        if (data['RSI'].iloc[i] <= 40 and data['Close'].iloc[i] > data['Lower_BB'].iloc[i]) and (data['Close'].iloc[i-1] <= data['Lower_BB'].iloc[i-1]):
            data.at[data.index[i], 'Signals'] = 1
        else:
            data.at[data.index[i], 'Signals'] = 0
        
    return data

us500 = generate_signal(us500)
us500.dropna(inplace=True)
us500


,Open,High,Low,Close,Tick_volume,Spread,Real_volume,Lower_BB,RSI,Signals
time,,,,,,,,,,
2023-01-03 05:45:00,3844.01,3847.16,3843.89,3846.15,544,40,0,3824.137379,54.311163,0
2023-01-03 06:00:00,3846.39,3846.90,3842.89,3844.15,520,40,0,3825.001829,50.839111,0
2023-01-03 06:15:00,3844.27,3845.41,3843.64,3845.39,324,40,0,3825.338902,52.851631,0
2023-01-03 06:30:00,3845.66,3847.16,3844.89,3845.14,342,40,0,3825.706245,52.386003,0
2023-01-03 06:45:00,3845.39,3845.66,3843.39,3843.66,373,40,0,3826.201515,49.600081,0
...,...,...,...,...,...,...,...,...,...,...
2024-06-07 15:45:00,5328.64,5338.14,5324.89,5335.64,2216,46,0,5339.484347,29.236361,0
2024-06-07 16:00:00,5335.39,5345.89,5333.39,5341.89,1576,46,0,5337.510655,36.990828,1
2024-06-07 16:15:00,5341.89,5349.39,5341.64,5348.14,1612,46,0,5336.800311,43.641778,0


In [10]:


# Without RSI

def generate_signal(data):
    data['Lower_BB'] = ta.volatility.bollinger_lband(data['Close'], window=20)
    data['Signals'] = 0
    
    for i in range(1, len(data)):
        
        if (data['Close'].iloc[i] > data['Lower_BB'].iloc[i]) and (data['Close'].iloc[i-1] <= data['Lower_BB'].iloc[i-1]):
            data.at[data.index[i], 'Signals'] = 1
        else:
            data.at[data.index[i], 'Signals'] = 0
        
    return data

us500 = generate_signal(us500)
us500.dropna(inplace=True)
us500

,Open,High,Low,Close,Tick_volume,Spread,Real_volume,Lower_BB,RSI,Signals
time,,,,,,,,,,
2023-01-03 10:30:00,3866.40,3875.91,3864.64,3872.90,2992,40,0,3836.208780,72.190727,0
2023-01-03 10:45:00,3872.76,3876.41,3872.39,3874.64,2189,40,0,3834.838897,73.175236,0
2023-01-03 11:00:00,3874.90,3882.41,3873.91,3881.89,3391,40,0,3833.241389,76.852374,0
2023-01-03 11:15:00,3882.16,3885.16,3879.16,3882.15,2133,40,0,3832.493706,76.974275,0
2023-01-03 11:30:00,3882.14,3884.41,3880.89,3882.40,1650,40,0,3832.517081,77.099158,0
...,...,...,...,...,...,...,...,...,...,...
2024-06-07 15:45:00,5328.64,5338.14,5324.89,5335.64,2216,46,0,5339.484347,29.236361,0
2024-06-07 16:00:00,5335.39,5345.89,5333.39,5341.89,1576,46,0,5337.510655,36.990828,1
2024-06-07 16:15:00,5341.89,5349.39,5341.64,5348.14,1612,46,0,5336.800311,43.641778,0


In [11]:
us500['Signals'].sum()


1007

In [12]:
def SIGNAL():
    return us500.Signals

In [13]:


class BollingerBandBuy(Strategy):
    initsize = 5.0
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
    def next(self):
        super().next()
        if (self.signal1 > 0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0) :
            self.mysize = self.mysize*2
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            self.mysize = self.initsize
            
        if self.signal1 == 1:
            sl1 = self.data.Close[-1] - 250e-1
            tp1 = self.data.Close[-1] + 550e-1
            self.buy(sl = sl1, tp = tp1, size=self.mysize)      
            
bt = Backtest(us500, BollingerBandBuy, cash=10000, exclusive_orders=True, margin= 1/100)

output = bt.run()
output  


Start                     2023-01-03 10:30:00
End                       2024-06-07 16:45:00
Duration                    521 days 06:15:00
Exposure Time [%]                   86.123446
Equity Final [$]                     21278.75
Equity Peak [$]                       21455.9
Return [%]                           112.7875
Buy & Hold Return [%]                38.20109
Return (Ann.) [%]                   67.246397
Volatility (Ann.) [%]               104.12993
Sharpe Ratio                         0.645793
Sortino Ratio                        3.107353
Calmar Ratio                         1.691063
Max. Drawdown [%]                  -39.765745
Avg. Drawdown [%]                   -0.991587
Max. Drawdown Duration      185 days 22:00:00
Avg. Drawdown Duration        1 days 16:54:00
# Trades                                 1007
Win Rate [%]                        35.451837
Best Trade [%]                       1.438046
Worst Trade [%]                     -0.655466
Avg. Trade [%]                    

In [14]:
bt.plot()

c:\Users\User\anaconda3\anaconda_3\lib\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


ValueError: Length of values (2) does not match length of index (1)